In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

开始使用Vertex AI超参数调整管道组件

*注意：此笔记本使用KFP 1.x和GCPC 1.x。我们建议使用2.x版本*

## 概览

此教程演示了如何使用Vertex AI超参数调整管道组件。

了解更多关于[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)和[Vertex AI超参数调整](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)。

### 目标

在本教程中，您将学习如何使用预构建的Google Cloud管道组件来进行Vertex AI超参数调整。

本教程使用以下Google Cloud ML服务：

- Google Cloud管道组件
- Vertex AI数据集、模型和端点资源
- Vertex AI超参数调整

执行的步骤包括：

- 构建一个管道，用于：
    - 超参数调整/训练自定义模型。
    - 检索经过调整的超参数值和指标以进行优化。
    - 如果指标超过指定阈值。
      - 获取经过最佳调整的模型的模型构件的位置。
      - 将模型构件上传到Vertex AI模型资源。
- 执行一个Vertex AI管道。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)的[马和人类](https://www.tensorflow.org/datasets/catalog/horses_or_humans)。训练模型会预测图像是马还是人类。

费用

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

详细了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)根据您预期的使用情况生成费用估算。

安装所需的包以执行笔记本。

In [ ]:
! pip3 install -U tensorflow==2.5 -q
! pip3 install -U tensorflow-data-validation==1.2  -q
! pip3 install -U tensorflow-transform==1.2  -q
! pip3 install -U tensorflow-io==0.18  -q
! pip3 install --upgrade google-cloud-aiplatform[tensorboard] -q
! pip3 install --upgrade 'google-cloud-pipeline-components<2'  -q
! pip3 install --upgrade 'kfp<2' -q

仅 Colab 使用：请取消下面单元格的注释以重新启动核心

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 在开始之前

#### 设置您的项目 ID

**如果您不知道您的项目 ID**，请尝试以下方法：
- 运行 `gcloud config list`
- 运行 `gcloud projects list`
- 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

区域

您还可以更改由 Vertex AI 使用的 `REGION` 变量。
了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动进行验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
- 无需操作，因为您已经通过验证。

**2. 本地JupyterLab实例**，取消注释并运行。

In [ ]:
# ! gcloud auth login

3. 协作,取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何将云存储权限授予您的服务帐号，请访问https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

### 创建一个云存储桶

创建一个存储桶来存储诸如数据集等中间文件。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

#### 服务账户

**如果你不知道你的服务账户**，请尝试使用`gcloud`命令在下面执行第二个单元格来获取你的服务账户。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

## 为Vertex AI Pipelines设置服务账户访问权限

运行以下命令，将您的服务账户访问权限授予读取和写入管道工件的存储桶 -- 您只需要对每个服务账户运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

In [ ]:
import json

from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component

#### 导入TensorFlow

将TensorFlow包导入到你的Python环境中。

In [ ]:
import tensorflow as tf

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化用于 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像，以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个支持 GPU 的容器映像，每个 VM 分配 4 个 Nvidia Telsa K80 GPU，您应该指定：

（aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4）

否则，指定`(None, None)`以使用一个在 CPU 上运行的容器映像。

了解更多关于[您位置的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：TF 2.3 之前的 GPU 支持版本会在这个教程中加载自定义模型时失败。这是一个已知问题，在 TF 2.3 中已修复。这是由在服务函数中生成的静态图操作引起的。如果您在自定义模型上遇到这个问题，请使用具有 GPU 支持的 TF 2.3 容器映像。

In [ ]:
import os

TRAIN_GPU, TRAIN_NGPU = (None, None)
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置预先构建的容器

设置用于预测的预先构建的Docker容器镜像。

- 将变量`TF`设置为容器镜像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下列表显示了一些可用的预先构建的镜像：

要获取最新列表，请参见[用于预测的预先构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 的内存为 3.75GB。
     - `n1-highmem`: 每个 vCPU 的内存为 6.5GB。
     - `n1-highcpu`: 每个 vCPU 的内存为 0.9 GB。
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96 \] 中的数字

*注意：以下机器类型不支持训练:*

 - `standard`: 2 个 vCPUs
 - `highcpu`: 2, 4 和 8 个 vCPUs

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU*。

In [ ]:
TRAIN_COMPUTE = "n1-standard-4"
print("Train machine type", TRAIN_COMPUTE)

DEPLOY_COMPUTE = "n1-standard-4"
print("Deploy machine type", DEPLOY_COMPUTE)

### 检查调整包

#### 包布局

在开始调整之前，您将查看一个Python包是如何为自定义调整工作组装的。解压后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到Docker镜像操作环境的指令。

文件 `trainer/task.py` 是执行自定义调整工作的Python脚本。*注意*，当我们在工作池规范中引用它时，我们用点 (`trainer.task`) 替换目录斜杠，并删除文件后缀 (`.py`)。

#### 包装配

在接下来的步骤中，您将组装训练包。

In [ ]:
# Make folder for Python tuning script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow==2.5.0',\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Horses or Humans image classification\n\nVersion: 0.0.0\n\nSummary: Demostration tuning script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 创建Python培训套件的任务脚本

接下来，您会创建用于驱动培训套件的`task.py`脚本。一些值得注意的步骤包括：

- 命令行参数：
    - `model-dir`：用于保存训练模型的位置。在使用Vertex AI自定义训练时，该位置将在环境变量中指定为：`AIP_MODEL_DIR`，
    - `epochs`：训练的时期数。
    - `learning_rate`：学习速率的超参数。
    - `batch_size`：批量大小的超参数。


- 数据预处理（`get_data()`）
    - 加载并预处理数据集作为`tf.data.Dataset`生成器。


- 模型架构（`get_model()`）：
    - 构建对应的模型架构。


- 训练（`train_model()`）：
    - 训练模型。


- 模型工件保存
    - 保存模型工件的云存储位置。

In [ ]:
%%writefile custom/trainer/task.py
import os
os.system('pip install cloudml-hypertune')  # alternaterly, this can be added to the Dockerfile

import tensorflow as tf
import tensorflow_datasets as tfds
import argparse
import hypertune


def get_args():
  '''Parses args. Must include all hyperparameters you want to tune.'''

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--epochs',
      required=True,
      type=int,
      help='number of epochs')
  parser.add_argument(
      '--learning_rate',
      required=True,
      type=float,
      help='learning rate')
  parser.add_argument(
      '--momentum',
      required=False,
      type=float,
      default=0.5,
      help='SGD momentum value')
  parser.add_argument(
      '--batch_size',
      required=True,
      type=int,
      help='the batch size')
  parser.add_argument(
      '--model-dir',
      dest='model_dir',
      default=os.getenv('AIP_MODEL_DIR'),
      type=str, help='Model dir.')
  args = parser.parse_args()
  return args

def preprocess_data(image, label):
  '''Resizes and scales images.'''

  image = tf.image.resize(image, (150,150))
  return tf.cast(image, tf.float32) / 255., label


def get_data():
  '''Loads Horses Or Humans dataset and preprocesses data.'''

  data, info = tfds.load(name='horses_or_humans', as_supervised=True, with_info=True)

  # Create train dataset
  train_data = data['train'].map(preprocess_data)
  train_data  = train_data.shuffle(1000)
  train_data  = train_data.batch(64)

  # Create validation dataset
  validation_data = data['test'].map(preprocess_data)
  validation_data  = validation_data.batch(64)

  return train_data, validation_data


def get_model(learning_rate, momentum):
  '''Defines and complies model.'''

  inputs = tf.keras.Input(shape=(150, 150, 3))
  x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(inputs)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Flatten()(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
  model = tf.keras.Model(inputs, outputs)
  model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum),
      metrics=['accuracy'])
  return model

def train_model(model, train_data, validation_data, epochs, batch_size):

  history = model.fit(train_data, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

  # DEFINE METRIC
  hp_metric = history.history['val_accuracy'][-1]

  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=hp_metric,
      global_step=epochs
  )

  return model


def main():
  args = get_args()
  train_data, validation_data = get_data()

  model = get_model(args.learning_rate, args.momentum)

  model = train_model(model, train_data, validation_data, args.epochs, args.batch_size)

  model.save(args.model_dir)


if __name__ == "__main__":
    main()

将调优脚本存储在您的云存储桶中

接下来，将调优文件夹打包成压缩的tar文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_horses_or_humans.tar.gz

### 创建一个Docker文件

为了使用自己的自定义训练容器，请构建一个Docker文件，并将其嵌入到带有训练脚本的容器中。

#### 编写 Docker 文件内容

将您的代码容器化的第一步是创建一个 Docker 文件。在您的 Docker 文件中，您将包含运行容器映像所需的所有命令。它将安装您正在使用的所有库，并为训练代码设置入口点。

1. 从 TensorFlow 仓库中安装预定义的用于深度学习映像的容器映像。
2. 将 Python 训练代码复制进来，稍后会展示。
3. 将入口设置为 Python 训练脚本 `trainer/task.py`。注意，`ENTRYPOINT` 命令中省略了 `.py`，因为这是显而易见的。

In [ ]:
%%writefile custom/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3
WORKDIR /root

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

#### 在本地构建容器

接下来，在提交到Google容器注册表时，请提供一个您自定义容器的名称。

In [ ]:
TRAIN_IMAGE = "gcr.io/" + PROJECT_ID + "/horses_or_humans:v1"

接下来，建造容器。

In [ ]:
! docker build custom -t $TRAIN_IMAGE

#### 注册自定义容器

当您在本地运行完容器后，请将其推送至Google容器注册表。

In [ ]:
! docker push $TRAIN_IMAGE

### 构建超参数调整流水线

接下来，构建以下任务的流水线：

- 创建/执行一个超参数调整作业
- 获取所有试验结果
- 获取最佳试验结果
- 确定最佳试验结果是否超过一个阈值：
    - 检索超参数数值
    - 确定最佳模型的云存储位置
    - 将最佳模型上传为 Vertex AI 模型资源。

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/custom_icn_tuning".format(BUCKET_URI)


@component(packages_to_install=["google-cloud-aiplatform"])
def model_dir(base_output_directory: str, best_trial: str) -> str:
    from google.cloud.aiplatform_v1.types import study

    trial_proto = study.Trial.from_json(best_trial)
    model_id = trial_proto.id
    return f"{base_output_directory}/{model_id}/model"


@dsl.pipeline(
    name="hp-tuning", description="Custom image classification hyperparameter tuning"
)
def pipeline(
    display_name: str,
    worker_pool_specs: list,
    study_spec_metrics: list,
    study_spec_parameters: list,
    threshold: float,
    deploy_image: str,
    max_trial_count: int = 5,
    parallel_trial_count: int = 1,
    base_output_directory: str = PIPELINE_ROOT,
    labels: dict = {},
    project: str = PROJECT_ID,
    region: str = REGION,
):
    from google_cloud_pipeline_components.experimental import \
        hyperparameter_tuning_job
    from google_cloud_pipeline_components.types import artifact_types
    from google_cloud_pipeline_components.v1.hyperparameter_tuning_job import \
        HyperparameterTuningJobRunOp
    from google_cloud_pipeline_components.v1.model import ModelUploadOp
    from kfp.v2.components import importer_node

    tuning_op = HyperparameterTuningJobRunOp(
        display_name=display_name,
        project=project,
        location=region,
        worker_pool_specs=worker_pool_specs,
        study_spec_metrics=study_spec_metrics,
        study_spec_parameters=study_spec_parameters,
        max_trial_count=max_trial_count,
        parallel_trial_count=parallel_trial_count,
        base_output_directory=base_output_directory,
    )

    trials_op = hyperparameter_tuning_job.GetTrialsOp(
        gcp_resources=tuning_op.outputs["gcp_resources"]
    )

    best_trial_op = hyperparameter_tuning_job.GetBestTrialOp(
        trials=trials_op.output, study_spec_metrics=study_spec_metrics
    )

    threshold_op = hyperparameter_tuning_job.IsMetricBeyondThresholdOp(
        trial=best_trial_op.output,
        study_spec_metrics=study_spec_metrics,
        threshold=threshold,
    )

    with dsl.Condition(
        threshold_op.output == "true",
        name="deploy_decision",
    ):
        _ = hyperparameter_tuning_job.GetHyperparametersOp(trial=best_trial_op.output)

        model_dir_op = model_dir(base_output_directory, best_trial_op.output)

        import_unmanaged_model_op = importer_node.importer(
            artifact_uri=model_dir_op.output,
            artifact_class=artifact_types.UnmanagedContainerModel,
            metadata={
                "containerSpec": {
                    "imageUri": DEPLOY_IMAGE,
                },
            },
        ).after(model_dir_op)

        _ = ModelUploadOp(
            project=project,
            display_name=display_name,
            unmanaged_container_model=import_unmanaged_model_op.outputs["artifact"],
        ).after(import_unmanaged_model_op)

### 创建超参数调整规范

接下来，您按照以下方式构建工作池规范、研究的指标和参数规范：

**工作池规范**

此规范描述了执行超参数研究的机器和容器要求以及扩展性。由于训练模块嵌入在docker镜像中，您可以使用`args`字段来指定任何命令行参数，这些参数不是研究的一部分，传递给训练模块。在这个例子中，您传递了epoch的数量。

**参数规范**

此规范描述了要调整的超参数以及要调整的值范围。对于每个研究，这些参数的试验值作为命令行参数传递给训练模块，格式为`--<parameter_name>=<trial_value>`。

**指标规范**

此规范描述了在研究中要评估的指标或指标，以及该指标是最小化还是最大化。

In [ ]:
from google_cloud_pipeline_components.experimental import \
    hyperparameter_tuning_job

gpu = "ACCELERATOR_TYPE_UNSPECIFIED"
accelerator_count = 0

if TRAIN_GPU:
    gpu = TRAIN_GPU.name
    accelerator_count = 1

else:
    gpu = "ACCELERATOR_TYPE_UNSPECIFIED"
    accelerator_count = (
        0  # same problem with accelerator_count, if we keep is as "None" its not
    )

CMDARGS = [
    "--epochs=10",
]

# The spec of the worker pools including machine type and Docker image
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAIN_COMPUTE,
            "accelerator_type": gpu,
            "accelerator_count": accelerator_count,
        },
        "replica_count": 1,
        "container_spec": {"image_uri": TRAIN_IMAGE, "args": CMDARGS},
    }
]

# List serialized from the dictionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# and the dictionary value is the optimization goal of the metric.
metric_spec = hyperparameter_tuning_job.serialize_metrics({"accuracy": "maximize"})

# List serialized from the parameter dictionary. The dictionary
# represents parameters to optimize. The dictionary key is the parameter_id,
# which is passed into your training job as a command line key word argument, and the
# dictionary value is the parameter specification of the metric.
parameter_spec = hyperparameter_tuning_job.serialize_parameters(
    {
        "learning_rate": aiplatform.hyperparameter_tuning.DoubleParameterSpec(
            min=0.001, max=1, scale="log"
        ),
        "batch_size": aiplatform.hyperparameter_tuning.DiscreteParameterSpec(
            values=[16, 32, 64], scale=None
        ),
    }
)

### 编译和执行超参数调优管道

接下来，您编译管道，然后执行它。管道接受以下参数，这些参数作为字典`parameter_values`传递：

- `display_name`：管道作业的人类可读名称。
- `worker_pool_specs`：机器和容器，以及自动扩展要求，以及命令行参数。
- `study_spec_metrics`：优化研究试验中的度量标准。
- `study_spec_parameters`：要调整的参数。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="hp_tune_pipeline_job.json"
)

pipeline = aiplatform.PipelineJob(
    display_name="hp_tuning",
    template_path="hp_tune_pipeline_job.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "display_name": "hp_tuning",
        "worker_pool_specs": worker_pool_specs,
        "study_spec_metrics": metric_spec,
        "study_spec_parameters": parameter_spec,
        "threshold": 0.7,
        "deploy_image": DEPLOY_IMAGE,
    },
    enable_caching=False,
)

pipeline.run()

! rm -rf hp_tune_pipeline_job.json custom custom.tar.gz

查看数据管道执行结果

In [ ]:
PROJECT_NUMBER = pipeline.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        EVAL_METRICS = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/evaluation_metrics"
        )
        if tf.io.gfile.exists(EXECUTE_OUTPUT):
            ! gsutil cat $EXECUTE_OUTPUT
            return EXECUTE_OUTPUT
        elif tf.io.gfile.exists(GCP_RESOURCES):
            ! gsutil cat $GCP_RESOURCES
            return GCP_RESOURCES
        elif tf.io.gfile.exists(EVAL_METRICS):
            ! gsutil cat $EVAL_METRICS
            return EVAL_METRICS

    return None


print("hyperparameter-tuning-job")
artifacts = print_pipeline_output(pipeline, "hyperparameter-tuning-job")
print("\n\n")
print("gettrialsop")
artifacts = print_pipeline_output(pipeline, "gettrialsop")
print("\n\n")
print("getbesttrialop")
artifacts = print_pipeline_output(pipeline, "getbesttrialop")
print("\n\n")
output = !gsutil cat $artifacts
output = json.loads(output[0])
best_trial = json.loads(output["parameters"]["Output"]["stringValue"])
model_id = best_trial["id"]
print("BEST MODEL", model_id)
parameters = best_trial["parameters"]
batch_size = parameters[0]["value"]
print("BATCH SIZE", batch_size)
learning_rate = parameters[1]["value"]
print("LR", learning_rate)
MODEL_DIR = f"{PIPELINE_ROOT}/{model_id}/model"

print("ismetricbeyondthresholdop")
artifacts = print_pipeline_output(pipeline, "ismetricbeyondthresholdop")
print("\n\n")
print("deploy-decision")
artifacts = print_pipeline_output(pipeline, "deploy-decision")
print("\n\n")
print("model-dir")
artifacts = print_pipeline_output(pipeline, "model-dir")
print("\n\n")
print("model-upload")
artifacts = print_pipeline_output(pipeline, "model-upload")
print("\n\n")

删除流水线作业

在流水线作业完成后，您可以使用`delete()`方法删除流水线作业。在完成之前，可以使用`cancel()`方法取消流水线作业。

In [ ]:
pipeline.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的个别资源：

- Cloud Storage Bucket

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI